<a href="https://colab.research.google.com/github/wanga-dev/Coursera_Capstone/blob/main/SpacexDashFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly

     |████████████████████████████████| 75 kB 2.5 MB/s 
     |████████████████████████████████| 1.0 MB 11.7 MB/s 
     |████████████████████████████████| 3.5 MB 36.1 MB/s 
     |████████████████████████████████| 188 kB 50.3 MB/s 
     |████████████████████████████████| 1.8 MB 27.0 MB/s 
     |████████████████████████████████| 357 kB 49.7 MB/s 
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84021 sha256=9760deb0037161db1329f3b038a96802486b9f86c8f7cdd0fd7d7197154ed894
  Stored in directory: /root/.cache/pip/wheels/85/e9/f4/033d9bd19fe04ba8b3b41b6cfaec3c4634e85bdcdd82858ff3
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527013 sha256=83590241aa9e131de5bea4b6f54befdf70a7c3b5597ec638f72f4e62c7ede095
  Stored in directory: /root/.cache/pip/wheels/02/a5/de/374685270e68a4a49867f9c0ddd477a85e09eb1a2cb0998f92
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427844 sha25

In [3]:
from jupyter_dash import JupyterDash

In [4]:
# Import required libraries

import plotly.graph_objects as go
from dash import no_update

In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import plotly.express as px

# Read the airline data into pandas dataframe
#spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
#app = dash.Dash(__name__)

app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),
                                    html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(site):
    filtered_df=spacex_df
    if site=='ALL':
        fig=px.pie(spacex_df,values='class',
        names='Launch Site',
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df=spacex_df[spacex_df['Launch Site']== site]
        df1=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(df1, values='class count', names='class', title=f"Total Success Launches for site {site}")
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(site,payload):
    low, high = (payload[0],payload[1])
    mask=spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site=='ALL':
        fig=px.scatter(mask,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        mask_filtered=mask[mask['Launch Site']==site]
        fig=px.scatter(mask_filtered,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload Mass for' + site)
        return fig

# Run the app
if __name__ == '__main__':
    #app.run_server()
    app.run_server(mode="inline", host="localhost", debug=True, dev_tools_ui=False, dev_tools_props_check=False)

<IPython.core.display.Javascript object>

In [6]:
#Upload the SPaceX dataset developed in prior labs
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


--2021-11-29 03:28:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-11-29 03:28:24 (348 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [7]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
